In [1]:
import mongoengine
from mongoengine import *
from dbmodels import *
from readref import *

import pandas as pd
from bson.objectid import ObjectId
import configparser
import pickle

In [2]:
# Read the confidentials.
credentials = configparser.ConfigParser()
credentials.read('credentials.ini')

# Connect to the database.
connect(
    db = credentials.get('lb', 'db'),
    username = credentials.get('lb', 'username'),
    password = credentials.get('lb', 'password'),
    host = credentials.get('lb', 'host'), 
    port = int(credentials.get('lb', 'port')), 
);

# Get reference

Get references from publications.

What I want to get:

```
{article1: [ref1, ref2, ...], 
 article2: [ref1, ref2, ...], 
 ...}

{book1: [ref1, ref2, ...], 
 book2: [ref1, ref2, ...], 
 ...}
```

And everything is stored as its ID.

Different approaches:
- Iterate all refs: find the pub of ref
- Iterate all pubs: find all ref in the pub

`Problem: How many do we need?`

In [3]:
# From Publications to References

# For Articles:
article = dict()
limit_a = 1000
for a in Article.objects():
    rs = Reference.objects(document_id=a.document_id
                          ,start_img_number__gte=a.start_img_number
                          ,end_img_number__lte=a.end_img_number)
    if len(rs) == 0: continue
    article[a.id] = [r.id for r in rs]
    if len(article) == limit_a: break

# For Books:
book = dict()
limit_b = 100
for b in Book.objects():
    rs = Reference.objects(document_id=b.document_id)
    if len(rs) == 0: continue
    book[b.id] = [r.id for r in rs]
    if len(book) == limit_b: break

In [4]:
# Iterate all articles
ref_of_article = dict()
for a, rs in article.items():
    ref_of_article[a] = read_refs_publication(rs)

# Print out in the order of reference
i = 0  # only the first three
for a, rs in ref_of_article.items():
    print("*****{}*****".format(a))
    for key in sorted(rs.keys()):
        print("{}: {}".format(key, rs[key]))
    print("\n")
    i = i + 1
    if i==3: break

pd.Series(ref_of_article).to_pickle('./pickle/local/ref_article.pickle')

*****595f9d41fe7683316b2dc6f3*****
65001: {'rid': ObjectId('58e4ce00fe7683152b56de80'), 'surface': 'COMUNICAZIONI LA PUSTERIA E LE MIGRAZIONI BARBARICHE NELLE CONSIDERAZIONI DI UN LINGUISTA Sulle lotte combattute fra gli Slavi carantani e i Baiuvari per Γ assestamento nelle Alpi orientali, P unica fonte storica di cui di- sponiamo è 1’«Historia Langobardorum» di Paolo Diacono,', 'author': ['COMUNICAZIONI LA PUSTERIA E LE MIGRAZIONI BARBARICHE NELLE CONSIDERAZIONI DI UN LINGUISTA'], 'title': ['Sulle lotte combattute fra gli Slavi carantani e i Baiuvari per Γ assestamento nelle Alpi orientali, P unica fonte storica di cui di- sponiamo è 1’«Historia Langobardorum» di Paolo Diacono,']}
67001: {'rid': ObjectId('58e4ce00fe7683152b56de81'), 'surface': 'profondità negli strati toponomastici, quali risultino le condizioni etnico-linguistiche della Pusteria nei secoli VI, VII, Vili in relazione: 1)', 'title': ['profondità negli strati toponomastici, quali risultino le condizioni etnico-linguisti

FileNotFoundError: [Errno 2] No such file or directory: './pickle/local/ref_article.pickle'

In [19]:
# Iterate all books
ref_of_book = dict()
for b, rs in book.items():
    ref_of_book[b] = read_refs_publication(rs)

# Print out in the order of reference
i = 0  # only the first three
for b, rs in ref_of_book.items():
    print("*****{}*****".format(b))
    for key in sorted(rs.keys()):
        print("{}: {}".format(key, rs[key]))
    print("\n")
    i = i + 1
    if i==3: break

pd.Series(ref_of_book).to_pickle('./pickle/local/ref_book.pickle')

*****595f9d26fe7683316b2dc5d7*****
3001: {'rid': ObjectId('5919e9673dde944752f6ebf7'), 'surface': 'Published by the Press Syndicate of the University of Cambridge The Pitt Building, Trumpington Street, Cambridge CB2 IRP\xa040 West 20th Street, New York, NY 10011, USA\xa010', 'author': ['Published by'], 'title': ['the Press Syndicate of the University of Cambridge The Pitt Building, Trumpington Street, Cambridge CB2 IRP\xa040 West 20th Street,'], 'year': ['10011, USA\xa010']}
3002: {'rid': ObjectId('5919e9673dde944752f6ebf8'), 'surface': 'Stamford Road, Oakleigh, Melbourne 3166, Australia © Cambridge University Press 1991', 'author': ['Stamford Road, Oakleigh,'], 'title': ['Melbourne 3166,'], 'year': ['1991']}
3003: {'rid': ObjectId('5919e9673dde944752f6ebf9'), 'surface': 'the United States of America Library of Congress Cataloging-in-Publication Data Shulz, Anne Markham, 1938— Giambattista and Lorenzo Bregno : Venetian sculpture in the high renaissance / Anne Markham Schulz,\xa0p cm. I

134009: {'rid': ObjectId('5919e9673dde944752f6eec6'), 'surface': 'Sartori/Luisetto, 1983, p 362, doc. 467 XVIII.', 'author': ['Sartori/Luisetto,'], 'year': ['1983,']}
134010: {'rid': ObjectId('5919e9673dde944752f6eec7'), 'surface': 'GIAMBATTISTA BREGNO, ANCELS FROM S. MARIA DEI SERVI, VENICE A.Payment to Bartolomeo Garbin for polishing\xa0the Anejéis, July 30,', 'author': ['GIAMBATTISTA BREGNO, ANCELS FROM S. MARIA DEI SERVI, VENICE'], 'title': ['A.Payment to Bartolomeo Garbin for polishing\xa0the Anejéis, July']}
134011: {'rid': ObjectId('5919e9673dde944752f6eec8'), 'surface': 'ASV, Procuratori di S. Marco, Commissarie, Misti, Busta 98A, filza 1523-24, c. 11'}
134012: {'rid': ObjectId('5919e9673dde944752f6eec9'), 'surface': 'left 1524 dito [30 luio] contadj a bortolomio garbín segador per zornade 5 fate questa settimana in fregar i anzollj et renovarllj. Fregar Ia figura de la sepoltura et altro Risarete a soldi 16 al dj monta a ........... ............................lire 4, soldi. .

196005: {'rid': ObjectId('5919e9673dde944752f6f3ad'), 'surface': 'Appendix A, Doc, Vili, A, B. 1.1For the Giustiniani Tomb, see Schulz, 1983, pp. 186—8, no. 23.', 'author': ['Appendix A,', 'Schulz,'], 'title': ['Doc,'], 'year': ['1983,']}
196006: {'rid': ObjectId('5919e9673dde944752f6f3ae'), 'surface': 'Correr, MS Gradenigo 228/ I, Grevembroch, i, 1754, с.92 3.1Jestaz, 1986, p. 156,', 'year': ['1754, с.92 3.1Jestaz, 1986,']}
196007: {'rid': ObjectId('5919e9673dde944752f6f3af'), 'surface': 'contrast to the Giustiniani\xa0figures, corner figures of the Zen Tomb are not set on a diagonal,\xa0but rather are aligned with the lateral faces of the sacrophagus. 21Zanotto, in Diedo/Zanotto, [1839], n.p. 22Stringa, 1610, pp. I7vf. 23Temanza, 1778, pp. 88f, 114.', 'title': ['contrast to the Giustiniani\xa0figures, corner figures of the Zen Tomb are not set on a diagonal,\xa0but rather are aligned with the lateral faces of the sacrophagus. 21Zanotto, in Diedo/Zanotto,'], 'year': ['[1839],']}
19600

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



275009: {'rid': ObjectId('5919e9663dde944752f6343c'), 'surface': 'C.X, Misti, reg. 16, c. 187r.'}
275010: {'rid': ObjectId('5919e9663dde944752f6343d'), 'surface': 'Ibid., c. 192r.', 'abbre': ['Ibid., c.'], 'abbre_pos': [1]}
275011: {'rid': ObjectId('5919e9663dde944752f6343e'), 'surface': 'Ibid., cc. 2r e 3v, 2 e 18 marzo 1461.', 'abbre': ['Ibid.,'], 'abbre_pos': [1]}
275012: {'rid': ObjectId('5919e9663dde944752f6343f'), 'surface': 'Ibid., c. 62r, 12 maggio 1462.', 'abbre': ['Ibid.,'], 'abbre_pos': [1]}
275013: {'rid': ObjectId('5919e9663dde944752f63440'), 'surface': 'l’assegnazione dei beni dei ribelli a sudditi che invece si erano dimostrati fedeli alla Repubblica, poteva costituire un importante strumento\xa0di legittimazione del potere e di controllo sulla riottosa nobiltà di terraferma.\xa0Sono queste necessità politiche che determinano l’intervento dei Dieci.', 'title': ['l’assegnazione dei beni dei ribelli a sudditi che invece si erano dimostrati fedeli alla Repubblica, poteva co

In [20]:
# Load
d_a = pd.read_pickle('./pickle/local/ref_article.pickle')
d_b = pd.read_pickle('./pickle/local/ref_book.pickle')

# Get refernces from publications with particial references

In [100]:
list_pub = []
for r in Reference.objects():
    p = r.get_containing_publication()
    if p in list_pub: continue
        
    for v in r.contents.values():
        if "abbreviation" in v.values():
            list_pub.append(p)
            break
    if len(list_pub)==500: break

KeyboardInterrupt: 

In [102]:
len(list_pub)

427

In [91]:
def ref_article(a):
    refs = Reference.objects(document_id=a.document_id,
                             start_img_number__gte=a.start_img_number,
                             end_img_number__lte=a.end_img_number)
    refs_id = [r.id for r in refs]
    return pd.DataFrame.from_dict(read_refs_publication(refs_id), orient='index')

In [105]:
df = ref_article(list_pub[80])

/Users/suntao/anaconda3/envs/lb/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: generator 'QuerySet._iter_results' raised StopIteration


In [116]:
for r in df.surface.sort_values().values:
    print(r)

A. Serena, La cultura umanistica a Treviso nel secolo XV, Venezia, 1912, p. 308.
A. Toffoli, Immagini di Vittorio Veneto, Udine, 1970, pp. 122-136;
Affreschi di Nicolò di Pietro e di Jacobello ! del Fiore a Serravalle, in «Rivista d’Arte», 1955, p. 157;
Antonio Zago, evidentemente un discepolo di Francesco Squarcione.
B. Be- I' RENSON, Italian pictures of the Renaissance: Venetian School, I, Londra 1957, p. 206;
B. Paiutta, La chiesa di San Giovanni Battista a Vittorio Veneto, Vit- torio Veneto, 1957, p. 60;
Barbiero, Le Confraternite del SS. Sacramento prima del 1539, Vedelago, 1941, p. 32;
Battistella, La Repubblica di Venezia . .., Venezia, 1921, p. 322
Battistella, Op. cit., 1921, p. 347.
Berenson, Op. cit., 1957, p. 93
Bettini, Giusto de' Menabuoi nel Battistero del Duomo di Padova, Venezia, 1960, pp. 20.
C. Huter, The Ceneda Master, in «Arte Veneta», XXVII, 1973, p. 25;
C. L. , Ragghianti, Stefano da Ferrara, Firenze, 1972, p. 176, nota 33.
Casta- gno, Filippo Lippi, Donatello e 